# InSAR coherence timeseries

This notebook demonstrates how to generate InSAR coherence time series for all Sentinel-1 bursts defined in a given GeoJSON file (e.g., `s1_bursts_glacierSouthTyrol.geojson`). The GeoJSON file is a filtered subset of the Burst ID map available from the SAR-MPC products (https://sar-mpc.eu/sar-mpc-products/), containing information about subswath name and burst ID for each burst.

In this example, we generate a time series of 12 days InSAR coherence for the summer months of 2023, using all Sentinel-1 bursts that cover the glaciers of South Tyrol.

In [1]:
import openeo
import geopandas as gpd
import os

import urllib
import json
from datetime import datetime

## Input setting and preparation

In [2]:
start_date, end_date = ["2023-07-01", "2023-09-30"]

output_folder = '/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Case_studies/InSAR_coherence_glaciers/coh_timseries/'

bursts_filename = 's1_bursts_glacierSouthTyrol.geojson'
bursts = gpd.read_file(bursts_filename)

In [3]:
url = "https://openeo.dataspace.copernicus.eu"
connection = openeo.connect(url).authenticate_oidc()

Authenticated using refresh token.


In [4]:
def create_InSARpairs(burst_id, subswath_id, temporal_baseline):

    https_request = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$filter=" + urllib.parse.quote(
        f"ContentDate/Start ge {start_date}T00:00:00.000Z and ContentDate/Start le {end_date}T23:59:59.000Z and "
        f"PolarisationChannels eq 'VV' and "
        f"BurstId eq {burst_id} and SwathIdentifier eq '{subswath_id}'"
    ) + "&$top=1000"
    
    with urllib.request.urlopen(https_request) as response:
        content = response.read().decode()
        s1_bursts = json.loads(content)

    dates = [datetime.strptime(b['BeginningDateTime'][:10], "%Y-%m-%d") for b in s1_bursts['value']]
    dates.sort()
    InSARpairs = []
    for date_ref in dates:
        for date_sec in dates:
            if (date_ref - date_sec).days == -temporal_baseline:
                InSARpairs.append([
                    datetime.strftime(date_ref, "%Y-%m-%d"),
                    datetime.strftime(date_sec, "%Y-%m-%d")
                ])

    return InSARpairs

## Create and start all the jobs (one job for each burst)

SAR coherence parallel processing with InSAR pair list as input

In [5]:
#b = bursts.iloc[0]
for idx, b in bursts.iterrows():

    datacube = connection.datacube_from_process(
        process_id="sar_coherence_parallel",
        InSAR_pairs=create_InSARpairs(int(b['burst_id']), b['subswath_name'], 12),
        burst_id=int(b['burst_id']),
        polarization="vv",
        sub_swath=b['subswath_name'],
        coherence_window_rg=21,
        coherence_window_az=5,
    )
    datacube = datacube.save_result(format='GTiff')

    job = datacube.create_job(
        title=f'sar_coherence_T{b['relative_orbit_number']}_{b['subswath_name']}_{b['burst_id']}',
        job_options={"image-name": "python311-dev"}
    )
    job.start()
    break

SAR coherence processing (no parallel) with temporal extent as input

In [8]:
for idx, b in bursts.iterrows():

    datacube = connection.datacube_from_process(
        process_id="sar_coherence",
        temporal_extent=[start_date, end_date],
        temporal_baseline=12,
        burst_id=int(b['burst_id']),
        polarization="vv",
        sub_swath=b['subswath_name'],
        coherence_window_rg=21,
        coherence_window_az=5,
    )
    datacube = datacube.save_result(format='GTiff')

    job = datacube.create_job(
        title=f'sar_coherence_T{b['relative_orbit_number']}_{b['subswath_name']}_{b['burst_id']}'
    )
    job.start()

Check job status

In [14]:
int(b['burst_id'])

30343

In [6]:
connection.list_jobs()

[{'created': '2025-12-05T10:35:06Z',
  'id': 'j-2512051035064735a3560458aa6f0f37',
  'progress': 0,
  'status': 'created',
  'title': 'sar_coherence_T15_IW3_30343',
  'updated': '2025-12-05T10:35:06Z'},
 {'created': '2025-12-05T09:02:28Z',
  'id': 'j-2512050902284f59b6a84512b536b5e5',
  'status': 'error',
  'title': 'sar_coherence_T15_IW3_30343',
  'updated': '2025-12-05T09:50:38Z'},
 {'created': '2025-12-03T16:04:15Z',
  'id': 'j-251203160415431a859b3404cd92bbab',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence_IW3_30343',
  'updated': '2025-12-03T16:16:18Z'},
 {'created': '2025-12-03T14:50:24Z',
  'id': 'j-2512031450244da79b5cd3ad40eb6d19',
  'status': 'error',
  'title': 'sar_coherence_IW3_30343',
  'updated': '2025-12-03T15:40:14Z'},
 {'created': '2025-12-03T14:34:20Z',
  'id': 'j-2512031434204bfdb585c3f150d662ba',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence',
  'updated': '2025-12-03T14:43:18Z'},
 {'created': '2025-11-28T15:35:27Z',
  'id': 'j-2511281535274e3d96f9eac248e9f259',
  'status': 'error',
  'title': 'sar_coherence_IW3_30343',
  'updated': '2025-11-28T16:29:15Z'},
 {'created': '2025-11-28T14:01:09Z',
  'id': 'j-2511281401094e97b16e48297d333f05',
  'status': 'error',
  'title': 'sar_coherence_IW3_30343',
  'updated': '2025-11-28T14:16:11Z'},
 {'created': '2025-11-25T16:24:56Z',
  'id': 'j-25112516245644d2b4fc00ed876317ee',
  'status': 'error',
  'title': 'sar_coherence',
  'updated': '2025-11-25T16:27:22Z'},
 {'created': '2025-11-25T14:03:06Z',
  'id': 'j-251125140306446d85c0ac3232dc2d6f',
  'progress': 100,
  'status': 'finished',
  'title': 'sar_coherence',
  'updated': '2025-11-25T14:39:50Z'},
 {'created': '2025-11-24T13:41:52Z',
  'id': 'j-25112413415240b5a37fb7ccc8779a9e',
  'status': 'error',
  'title': 'slc_preprocessing',
  'updated': '2025-11-24T13:44:12Z'},
 {'created': '2025-11-24T13:33:04Z',
  'id': 'j-2511241333044e1ea135979950ce688a',
  'status': 'error',
  'title': 'slc_preprocessing',
  'updated': '2025-11-24T13:35:20Z'},
 {'created': '2025-11-24T13:26:03Z',
  'id': 'j-2511241326034c868a51ab48c1bf74b3',
  'status': 'error',
  'title': 'slc_preprocessing',
  'updated': '2025-11-24T13:29:57Z'},
 {'created': '2025-11-24T13:25:30Z',
  'id': 'j-251124132530458b9592141b01070023',
  'progress': 0,
  'status': 'created',
  'title': 'slc_preprocessing',
  'updated': '2025-11-24T13:25:30Z'},
 {'created': '2025-10-07T06:55:30Z',
  'id': 'j-2510070655304021af4830435e55b5fd',
  'progress': 100,
  'status': 'finished',
  'title': 'InSAR_interferogram_unwrapping',
  'updated': '2025-10-07T07:07:49Z'},
 {'created': '2025-10-06T07:30:45Z',
  'id': 'j-251006073045411ba43a036bd90fdd63',
  'status': 'error',
  'title': 'InSAR_interferogram_unwrapping',
  'updated': '2025-10-06T07:45:13Z'},
 {'created': '2025-10-02T13:53:58Z',
  'id': 'j-2510021353584266bd112569e0b7cb04',
  'progress': 100,
  'status': 'finished',
  'title': 'InSAR_coherence',
  'updated': '2025-10-02T13:58:15Z'},
 {'created': '2025-10-02T13:44:05Z',
  'id': 'j-2510021344054367ad4df352a9439dba',
  'status': 'error',
  'title': 'InSAR_coherence',
  'updated': '2025-10-02T13:51:15Z'},
 {'created': '2025-10-02T13:35:19Z',
  'id': 'j-25100213351942289414f3ffcb52003b',
  'progress': 100,
  'status': 'finished',
  'title': 'InSAR_coherence',
  'updated': '2025-10-02T13:39:15Z'},
 {'created': '2025-10-02T13:25:21Z',
  'id': 'j-2510021325214f22881ac8d2fc5d7647',
  'status': 'error',
  'title': 'InSAR_coherence',
  'updated': '2025-10-02T13:32:14Z'},
 {'created': '2025-10-02T13:24:29Z',
  'id': 'j-2510021324294c37b6031d3e744a0d3e',
  'progress': 100,
  'status': 'finished',
  'title': 'InSAR_interferogram_unwrapping',
  'updated': '2025-10-02T13:32:15Z'},
 {'created': '2025-10-02T12:49:55Z',
  'id': 'j-251002124955417e806f59061e5049fa',
  'status': 'error',
  'title': 'InSAR_interferogram_unwrapping',
  'updated': '2025-10-02T13:05:12Z'},
 {'created': '2025-10-02T11:36:10Z',
  'id': 'j-2510021136104fa1830e1d581888d3

In [11]:
job_id = 'j-2512051035064735a3560458aa6f0f37'

In [12]:
connection.job(job_id)

<BatchJob job_id='j-2512051035064735a3560458aa6f0f37'>

In [9]:
connection.job(job_id).status()

'error'

In [13]:
connection.job(job_id).logs()

[{'id': '[1764931262189, 695098]',
  'time': '2025-12-05T10:41:02.189Z',
  'level': 'info',
  'message': 'Starting batch job os.getpid()=85: start 2025-12-05 10:41:02.189070'},
 {'id': '[1764931262189, 975359]',
  'time': '2025-12-05T10:41:02.189Z',
  'level': 'info',
  'message': "batch_job.py sys.version='3.11.7 (main, Aug 23 2024, 00:00:00) [GCC 11.4.1 20231218 (Red Hat 11.4.1-3)]'"},
 {'id': '[1764931262537, 149584]',
  'time': '2025-12-05T10:41:02.537Z',
  'level': 'info',
  'message': 'Loading custom processes from /opt/openeo-geopyspark-k8s-custom-processes/src/openeo_geopyspark_k8s_custom_processes/custom_processes.py'},
 {'id': '[1764931262538, 986943]',
  'time': '2025-12-05T10:41:02.538Z',
  'level': 'info',
  'message': 'Overriding process sar_backscatter (namespace backend)'},
 {'id': '[1764931262539, 451686]',
  'time': '2025-12-05T10:41:02.539Z',
  'level': 'info',
  'message': "load_custom_processes: exec loaded '/opt/openeo-geopyspark-k8s-custom-processes/src/openeo_geopyspark_k8s_custom_processes/custom_processes.py'"},
 {'id': '[1764931262539, 757372]',
  'time': '2025-12-05T10:41:02.539Z',
  'level': 'info',
  'message': 'Overriding process sar_backscatter (namespace backend)'},
 {'id': '[1764931290886, 902034]',
  'time': '2025-12-05T10:41:30.886Z',
  'level': 'info',
  'message': 'Job spec: {\n "process_graph": {\n  "sarcoherenceparallel1": {\n   "process_id": "sar_coherence_parallel",\n   "arguments": {\n    "InSAR_pairs": [\n     [\n      "2023-07-03",\n      "2023-07-15"\n     ],\n     [\n      "2023-07-15",\n      "2023-07-27"\n     ],\n     [\n      "2023-07-27",\n      "2023-08-08"\n     ],\n     [\n      "2023-08-08",\n      "2023-08-20"\n     ],\n     [\n      "2023-08-20",\n      "2023-09-01"\n     ],\n     [\n      "2023-09-01",\n      "2023-09-13"\n     ],\n     [\n      "2023-09-13",\n      "2023-09-25"\n     ]\n    ],\n    "burst_id": 30343,\n    "coherence_window_az": 5,\n    "coherence_window_rg": 21,\n    "polarization": "vv",\n    "sub_swath": "IW3"\n   }\n  },\n  "saveresult1": {\n   "process_id": "save_result",\n   "arguments": {\n    "data": {\n     "from_node": "sarcoherenceparallel1"\n    },\n    "format": "GTiff",\n    "options": {}\n   },\n   "result": true\n  }\n },\n "job_options": {\n  "image-name": "python311-dev",\n  "log_level": "info"\n }\n}'},
 {'id': '[1764931290974, 886110]',
  'time': '2025-12-05T10:41:30.974Z',
  'level': 'info',
  'message': "ZooKeeperUserDefinedProcessRepository with self._root='/openeo/udps' self._zk_client_reuse=True"},
 {'id': '[1764931292780, 878374]',
  'time': '2025-12-05T10:41:32.780Z',
  'level': 'warning',
  'message': 'Failed to enrich collection metadata of COPERNICUS_VEGETATION_INDICES: 404 Client Error: Not Found for url: https://stac.openeo.vito.be/collections/copernicus_r_utm-wgs84_10_m_hrvpp-vi_p_2017-now_v01'},
 {'id': '[1764931293118, 96014]',
  'time': '2025-12-05T10:41:33.118Z',
  'level': 'warning',
  'message': 'No elastic_job_registry given to GeoPySparkBackendImplementation, creating one'},
 {'id': '[1764931293319, 653780]',
  'time': '2025-12-05T10:41:33.319Z',
  'level': 'info',
  'message': "Loaded config config_id='os_creodias_openeo_k8s_prod' from config_path='/opt/backend_config/backendconfig.py' (reason='lazy_load')"},
 {'id': '[1764931293320, 689744]',
  'time': '2025-12-05T10:41:33.320Z',
  'level': 'warning',
  'message': "No 'openeo_api_version' in `evaluate()` env. Blindly assuming 1.2."},
 {'id': '[1764931293320, 862679]',
  'time': '2025-12-05T10:41:33.320Z',
  'level': 'info',
  'message': 'Doing dry run'},
 {'id': '[1764931293320, 975685]',
  'time': '2025-12-05T10:41:33.320Z',
  'level': 'info',
  'message': 'Correlation id: j-2512051035064735a3560458aa6f0f37'},
 {'id': '[1764931293619, 213789]',
  'time': '2025-12-05T10:41:33.619Z',
  'level': 'info',
  'message': "post_dry_run: post_dry_run_data={'global_extent_original': None, 'global_extent_aligned': None}"},
 {'id': '[1764931293619, 620505]',
  'time': '20

In [ ]:
connection.job(job_id).status().get_results().download_files(
    os.path.join(output_folder, connection.job(job_id).describe()['title'][14:])
)